# Join and clean ports data

#### Building a log of ship journeys, I found inconsistncies in the data
More specifically, the ports data set appears to be incomplete. Inspecting an intermediate test data set with the logs of one specific ship, I found that this specific ship (IMO = 9776224) frequently left from Felixstowe and came back to Felixstowe without passing by another port, which seems implausabile. After leaving from Felixstowe, I followed its coordinates and stopped where the speed got down to 0 for a couple of hours. I compared the coordinates where the ship was standing with the ports data set and could not find a match. In addition, I searched the coordinates where the ship was in google maps and found that it was standing in a port close to Kuala Lumpur. I compared with the ports data set again, and could not find a port in Kuala Lumpur in the data set. 

Hence, I downloaded a ports data set from the World Port Index (https://fgmod.nga.mil/apps/WPI-Viewer/) and will now merge the two port data sets that I have. 

In [1]:
import pandas as pd
import os
import numpy as np
import pycountry


In [2]:
# import both port files
current_directory = os.path.abspath(".")
relative_path = "data/PORTS.csv"
full_file_path = os.path.join(current_directory, relative_path)
ports_org = pd.read_csv(full_file_path)

relative_path = "data/WPI.csv"
full_file_path = os.path.join(current_directory, relative_path)
ports_wpi = pd.read_csv(full_file_path)
ports_wpi = ports_wpi[['Main Port Name', 'Country Code', 'Latitude', 'Longitude']]

In [3]:
ports_org[ports_org['PORT_NAME'] == 'HAMBURG']

,PORT_NAME,CNTR_CODE3,LONGITUDE,LATITUDE
4195,HAMBURG,DEU,9.964576,53.527793


In [4]:
ports_wpi[ports_wpi['Main Port Name'] == 'Hamburg']

,Main Port Name,Country Code,Latitude,Longitude
2813,Hamburg,Germany,53.55,9.933333


In [5]:
# create new column with port name is lower case latters to match the two dfs on it
ports_wpi['Port Name Lower'] = ports_wpi['Main Port Name'].str.lower()
ports_org['Port Name Lower'] = ports_org['PORT_NAME'].str.lower()

In [6]:
ports = pd.merge(ports_org, ports_wpi, how = 'outer', on = 'Port Name Lower', indicator = True)

In [7]:
ports = ports.sort_values(by = 'Port Name Lower')

In [8]:
# merge longitude and latidude in one column each
ports.loc[ports['LONGITUDE'].isna(), 'LONGITUDE'] = ports.loc[ports['LONGITUDE'].isna(), 'Longitude']
ports.loc[ports['LATITUDE'].isna(), 'LATITUDE'] = ports.loc[ports['LATITUDE'].isna(), 'Latitude']

In [9]:
# compute coordinate diff to previous entry in the data - 
# logic: If ports appear multiple times but have slighty different names, the difference to the previous entry will indicate two ports actually being the same
ports['lon diff'] = ports['LONGITUDE'] - ports['LONGITUDE'].shift(1)
ports['lat diff'] = ports['LATITUDE'] - ports['LATITUDE'].shift(1)
ports['position diff'] = abs(ports['lon diff']) + abs(ports['lat diff'])

In [10]:
print('This will eliminate', len(ports[ports['position diff'] <= 0.1]), 'ports')

This will eliminate 861 ports


In [11]:
ports_clean = ports[ports['position diff'] > 0.1]

In [12]:
# Merge country names - problem: I have the 3 digit codes and the full name of the country from the two data sets
# use pycountry to translate full country names to 3 digit code

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
    
countries['Turkey'] = 'TUR'
countries['Iran'] = 'IRN'
countries['South Korea'] = 'KOR'
countries['Congo (Brazzaville)'] = 'COG'
countries['Russia'] = 'RUS'
countries['Burma'] = 'BUR'

ports_clean['Country Code 3'] = ports_clean['Country Code'].map(countries)

C:\Users\tobia\AppData\Local\Temp\ipykernel_10560\2190878364.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ports_clean['Country Code 3'] = ports_clean['Country Code'].map(countries)


In [13]:
# impute all missing values in CNTR_CODE3 with country codes from other ports file
ports_clean.loc[ports_clean['CNTR_CODE3'].isna(), 'CNTR_CODE3'] = ports_clean.loc[ports_clean['CNTR_CODE3'].isna(), 'Country Code 3']

In [14]:
ports_clean[ports_clean['CNTR_CODE3'].isna()]

,PORT_NAME,CNTR_CODE3,LONGITUDE,LATITUDE,Port Name Lower,Main Port Name,Country Code,Latitude,Longitude,_merge,lon diff,lat diff,position diff,Country Code 3
8272,NaN,NaN,11.147972,63.717643,fiborgtangen,Fiborgtangen,,63.717643,11.147972,right_only,-17.952028,27.084310,45.036338,NaN
8211,NaN,NaN,6.333205,58.314907,jossingfjord,Jossingfjord,,58.314907,6.333205,right_only,71.183205,48.214907,119.398112,NaN
8266,NaN,NaN,27.835338,71.041347,mehamnfjorden,Mehamnfjorden,,71.041347,27.835338,right_only,-1.747995,34.908013,36.656008,NaN


In [18]:
# checked those three manually - they are in Norway. Impute NOR for those
ports_clean.loc[ports_clean['CNTR_CODE3'].isna(), 'CNTR_CODE3'] = 'NOR'

In [19]:
ports_clean[ports_clean['CNTR_CODE3'].isna()]

,PORT_NAME,CNTR_CODE3,LONGITUDE,LATITUDE,Port Name Lower,Main Port Name,Country Code,Latitude,Longitude,_merge,lon diff,lat diff,position diff,Country Code 3


In [20]:
ports_clean.to_csv('data/ports_clean.csv', index = False)